In [ ]:
import logging

from automata.cli.commands import reconfigure_logging
from automata.config.config_base import AgentConfigName
from automata.config.openai_config import OpenAIAutomataAgentConfigBuilder
from automata.agent.openai_agent import OpenAIAutomataAgent
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.tools.agent_tool_factory import AgentToolFactory

logger = logging.getLogger(__name__)
reconfigure_logging("DEBUG")

py_module_loader.initialize()

In [ ]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["document-oracle","py-reader"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)

In [ ]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

In [ ]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-3.5-turbo")
    .with_max_iterations(3)
    .build()
)

In [ ]:
import textwrap
instructions = textwrap.dedent('''
Provide a markdown python snippet, which can validly executed by `exec`. When ran, the snippet produces a valid instance of an openai agent config in the local variable `x`.
''')
agent = OpenAIAutomataAgent(instructions, config=agent_config)

In [ ]:
# Run the agent
result = agent.run()

In [ ]:
# Print the result
print(f"Result:\n{result}")

In [ ]:
result='```python\nfrom automata.config.openai_agent import OpenAIAutomataAgentConfig\n\nx = OpenAIAutomataAgentConfig()\nx.setup()```'
result='```python\n# We need to first import PyReader\nfrom automata.code_parsers.py.py_reader import PyReader\n\n# Next we create an instance of PyReader and assign it to the variable x\nx = PyReader()\n```'
cleaned_result = result.split('```python\n')[1].replace('```','')
print(cleaned_result)

In [ ]:
exec(cleaned_result)

In [ ]:
from automata.eval import (
    AgentEval,
    AgentEvalSetLoader,
    AgentEvaluationHarness,
    CodeWritingEval,
    OpenAIFunctionEval,
    CodeWritingAction
)
CodeWritingAction._extract_snippet(result)